In [421]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import *

# =================================
#             Functions
# =================================

def splitDate(arr):
    dateColumn = arr['Date'].str.split('-')
    arr['Year'] = pd.to_numeric(dateColumn.str[-3]);
    arr['Month'] = pd.to_numeric(dateColumn.str[-2]);
    arr['Week'] = pd.to_numeric(dateColumn.str[-1]) // 4;
    return arr

def speciesToNums(arr):
    arr.Species = arr.Species.map({'CULEX ERRATICUS':0, 'CULEX PIPIENS':1, 'CULEX PIPIENS/RESTUANS':2, 'CULEX RESTUANS':3, 'CULEX SALINARIUS':4, 'CULEX TARSALIS':5, 'CULEX TERRITANS':6, 'UNSPECIFIED CULEX':7})
    return arr

def join_features_weather(train, weather):
    joined = train.set_index('Date').join(weather.set_index('Date'), rsuffix = 'r')
    joined = joined[resArr]
    joined.index.name = None
    joined.index = range(0, len(joined))
    # специфичная обработка некоторых столбцов
    joined['Tavg'] = pd.to_numeric(joined['Tavg'])
    return joined

def our_cross_validation(train, trainAnswer):
#     разделяем выборку на две равные части - to do: попробовать cross_validation pandas
    dataTrain = train.ix[0:5000, ]
    dataTest = train.ix[5001:len(trainWeather), ]
    
    resTrain = trainAnswer.ix[0:5000, ]
    resTest = trainAnswer.ix[5001:len(train), ]
    
    # тренируем, смотрим качество на тренировочной
    XGBoost = xgb.XGBClassifier(max_depth = 2, n_estimators = 143)
    XGBoost.fit(dataTrain, resTrain)
    XGBoost.predict_proba(dataTrain)
    aucXGBTrain = metrics.roc_auc_score(resTrain, XGBoost.predict_proba(dataTrain)[:,1])
    print(aucXGBTrain)
    
    # смотрим качество на проверочной выборке
    aucXGBTest = metrics.roc_auc_score(resTest, XGBoost.predict_proba(dataTest)[:,1])
    print(aucXGBTest)
    
def get_answer_csv(train, trainAnswer, test, filePath):
    # тренировка и результат на тренировочной
    XGBoost = xgb.XGBClassifier(max_depth = 5, n_estimators = 200)
    XGBoost.fit(train, trainAnswer)
    aucXGBTrain = metrics.roc_auc_score(trainAnswer, XGBoost.predict_proba(train)[:,1])
    print(aucXGBTrain)

    # ответы для проверочной выборки и запись в файл
    temp = pd.Series(XGBoost.predict_proba(test)[:,1])
    temp.index = range(1, len(temp) + 1)
    temp.to_csv(filePath, header = True)
    

# =================================
#             First settings
# =================================
namesArray = ['Date', 'Species', 'Latitude', 'Longitude', 'Month', 'Week']

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Count norms
norm_1 = train[train['WnvPresent'] == 0]['WnvPresent'].count()
norm_2 = train[train['WnvPresent'] == 1]['WnvPresent'].count()

# Remove result column from train
trainAnswer = train['WnvPresent'];
train = train.drop('WnvPresent', axis = 1)

# Split date to year, month and week
train = splitDate(train)
test = splitDate(test)

# Convert species names to numbers
train = speciesToNums(train)
test = speciesToNums(test)

# ======== One-hot encoding example ========
# specie = pd.get_dummies(train.Species, prefix='Specie')
# train = train.join(specie)

# specieTest = pd.get_dummies(test.Species, prefix='Specie')
# test = test.join(specieTest)

# Get only columns from namesArray
train = train[namesArray]
test = test[namesArray]


In [422]:
# =================================
#             Weather
# =================================
weather = pd.read_csv('weather.csv')
# пока уберем станцию 1
weather.drop(weather[weather.Station == 1].index, inplace = True)
weatherNamesArray = ['Tavg', 'Tmin']

# Create main array from names arrays and remove date
resArr = namesArray + weatherNamesArray
resArr.remove('Date')

# Join train and test with weather
trainWeather = join_features_weather(train, weather);
testWeather = join_features_weather(test, weather);


In [423]:
# dataTrain = train.ix[0:5000, ]
# dataTest = train.ix[5001:10505, ]

# answerTrain = trainAnswer.ix[0:5000, ]
# answerTest = trainAnswer.ix[5001:10505, ]

In [424]:
# XGBoost.fit(dataTrain, answerTrain)
# XGBoost.predict_proba(dataTrain)
# aucXGBTrain = metrics.roc_auc_score(answerTrain, XGBoost.predict_proba(dataTrain)[:,1])

# print(aucXGBTrain)

# aucXGBTest = metrics.roc_auc_score(answerTest, XGBoost.predict_proba(dataTest)[:,1])

# print (XGBoost.predict_proba(dataTest)[:,1])

# print(aucXGBTest)

In [425]:
# Crossvalidation
our_cross_validation(trainWeather, trainAnswer)

# Get answer(print accuracy on train) and create csv
get_answer_csv(trainWeather, trainAnswer, testWeather, "submission-2.csv")

0.880674650713
0.76499696018
0.939492507573
